In [8]:
import numpy as np
from tempfile import TemporaryDirectory
from mf6.voronoiplus import TriangleGrid as Triangle
from shapely import Polygon
from mf6.voronoiplus import VoronoiGridPlus as Vor
from pathlib import Path
from mf6.simplemodel import SimpleModel
from mf6.headsplus import HeadsPlus as hp
import mf6.mfsimbase as mf
import shapely as shp

ModuleNotFoundError: No module named 'voronoiplus'

In [13]:
np.linspace(1,5,5
            ).tolist()

[1.0, 2.0, 3.0, 4.0, 5.0]

In [4]:
"""Example simple model"""
tri = Triangle(
    model_ws=Path.cwd(),
    angle=30
)
tri.add_circle(radius=100, center_coords=(50, 200), )
tri.add_circle(radius=5, center_coords=(50, 200), )
tri.add_circle(radius=5, center_coords=(50, 150), )
tri.add_regions(((50, 150), (0, 200)),
                maximum_areas=(0.5, 10))
tri.build()
# get bottom elevs
strike = 30  # Strike given in degrees from north
dip = 20  # Dip given in degrees from horizontal
known_point = (50, 200, 0)  # Known point (x, y, elevation)
pixel_size = 1  # Pixel size
bottom_raster_path = Path.cwd().joinpath('bottom_raster.tif')
top_raster_path = Path.cwd().joinpath('top_raster.tif')
vor = Vor(tri)
bottom_elevs = vor.get_raster_from_strike_dip(strike, dip, known_point, pixel_size, bottom_raster_path)
top_elevs = vor.get_raster_from_strike_dip(0, 0, (0,0,50), 1, top_raster_path)
vor = Vor(tri, rasters=[bottom_raster_path, top_raster_path])
nper = 31
center_cells = [32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56,
                57, 58, 59, 60, 61, 62, 63, 98, 123, 124, 163, 164, 165, 193, 194, 195, 196, 210, 211, 212, 213,
                234, 235, 236, 314, 337, 374, 375, 388, 392]
rch_trans = [np.random.random() * 20 for per in range(nper)]
rch_dict = {}
for per in range(nper):
    rch_dict[per] = [[cell, rch_trans[per]] for cell in center_cells]
model = SimpleModel(
    vor,
    #bottom=bottom_elevs['elev'].to_list(),
    top=50,
    nper=nper,
    rch_dict=rch_dict
)
model.run_simulation()
hds = hp(hds_path=model.model_output_folder_path.joinpath('mf6_model.hds'), vor=vor)


VoronoiGrid initialized.
VoronoiGrid initialized.


NameError: name 'SimpleModel' is not defined

In [11]:
for stp_per in hds.kstpkper:
    hds.plot_choropleth(stp_per, zoom=19, plot_mounding=True)

In [ ]:
### ### DASH ### ###
### DASH app to store selected cell ids ###

import dash
from dash import dcc
from dash import html, Input, Output
import dash_bootstrap_components as dbc

app = dash.Dash()
app.layout = html.Div(
    [
        dbc.Row(
            dbc.Col(
                [
                    dcc.Graph(
                        figure=vor.plot_choropleth(),
                        className="flex-grow-1",
                        style={"height": "95vh"},
                        id="fig",
                    )
                ],
                class_name="h-100 d-flex flex-column",
                style={"height": "95vh"},
            ),
            style={"height": "95vh"},
        ),
        dcc.Store(id="selected"),
    ],
    style={"height": "95vh"},
)


@app.callback(
    Output(component_id="selected", component_property="data"),
    Input(component_id="fig", component_property="selectedData"),
    prevent_initial_callbacks=True,
)
def on_select(selectedData):
    selected_cells = []
    for cell in selectedData["points"]:
        selected_cells.append(int(cell["location"]))
        print(selected_cells)
    return selected_cells


app.run(debug=True, use_reloader=False)  # Turn off reloader if inside Jupyter

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [15]:
from budgetplus import BudgetPlus
import paths
bud = BudgetPlus(cbb_path=model.model_output_folder_path.joinpath('mf6_model.cbc'), vor=vor)

In [25]:
rch_bud = bud.get_rch_bud()
rch_bud = (rch_bud.reset_index(drop=True))

In [46]:
rch_sum = rch_bud.loc[:,'q'].dropna().sum()